In [1]:
#Se cargan todas las librerías que vamos a usar

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize#Se cargan todas las librerías que vamos a usar



In [2]:
#Lectura de tablas de datos con panda
data = pd.read_csv('sentiment_tweets3.csv')
dataset = data['message to examine']
dataset[0:5]


0    just had a real good moment. i missssssssss hi...
1           is reading manga  http://plurk.com/p/mzp1e
2    @comeagainjen http://twitpic.com/2y2lx - http:...
3    @lapcat Need to send 'em to my accountant tomo...
4        ADD ME ON MYSPACE!!!  myspace.com/LookThunder
Name: message to examine, dtype: object

In [5]:
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
def limpiar_texto(data):

    for line in range(0,(len(data)-1)): 

        ## Eliminación de carácteres raros, URL, hashtags, menciones y emojis. 
        # Aparecían muchos carácteres hacia el final del dataset como . Buscando como podía eliminarlo, conseguí encontrar una
        # manera de recopilar esos puntos junto a otros caracteres como â a traves de: [^\x20-\x7E]. 
        new_line = re.sub(r'Ã|Â|[^\x20-\x7E]|ì|ë|°|ï|½|<Emoji:\s*[^>]+>|https?://\S+|www\.\S+|\b\S+\.com\S+\b|[@#]\w+|@|¦|¢',"", data[line])
        new2 = new_line.lower()
        tokenizer = TweetTokenizer()
        sep = tokenizer.tokenize(new2)
        #print(sep)
        stopW = set(stopwords.words('english'))
        #print(stopW)
        
        for lista in sep:
            filtered_sentence = [word for word in sep if not word in stopW]
            #print(filtered_sentence)
            
#        filtered_sentence = []
#        for word in sep:
#            if word not in stopW:
#                filtered_sentence.append(word)

            lemat = SnowballStemmer('english')
            stems = [lemat.stem(word) for word in filtered_sentence]
        #print(stems)
        final_text = ' '.join(stems)
        #print(final_text)
        data[line] == final_text 
        
    #print(len(data))
        
    return data

In [6]:
limpiar_texto(dataset)

0                   real good moment . missssssssss much ,
1                                               read manga
2                                                        -
3        need send ' em account tomorrow . odd , even r...
4                                         add myspac ! ! !
                               ...                        
10309    depress g herbo mood , i'm done stress peopl d...
10310    depress succumb brain make feel like never eno...
10311     ketamin nasal spray show promis depress , suicid
10312         dont mistak bad day depress ! everyon ' em !
10313                                                    0
Name: message to examine, Length: 10314, dtype: object

In [7]:
from textblob import TextBlob
def clasificador(data):
    labels = []
    for line in range(0,len(data)):
        text = TextBlob(data[line])
        sentiment_polarity = text.sentiment.polarity
        #print(sentiment_polarity, text)
        
        if -1 <= sentiment_polarity <= -0.8:
            label = "Hater"
        elif -0.8 < sentiment_polarity < 0:
            label = "Molesto"
        elif sentiment_polarity == 0:
            label = "Neutro"
        elif 0 < sentiment_polarity < 0.8:
            label = "Contento"
        elif 0.8 <= sentiment_polarity <= 1:
            label= "Muy feliz"
        labels.append(label)
        
#        print(line, text, sentiment_polarity, label)
#    print(labels)
#    print(len(labels))
#    print(len(data),len(labels))
    data = pd.DataFrame(data)
#    print(data[0:5])
    data['label'] = labels
#    data.insert(loc=1, column='label', value=labels)
    
#    print(data)
    return data


In [12]:
clasificador(dataset)

,message to examine,label
0,"real good moment . missssssssss much ,",Contento
1,read manga,Neutro
2,-,Neutro
3,"need send ' em account tomorrow . odd , even r...",Molesto
4,add myspac ! ! !,Neutro
...,...,...
10309,"depress g herbo mood , i'm done stress peopl d...",Molesto
10310,depress succumb brain make feel like never eno...,Molesto
10311,"ketamin nasal spray show promis depress , suicid",Molesto
10312,dont mistak bad day depress ! everyon ' em !,Molesto


In [9]:
dataset[0:5]


0               real good moment . missssssssss much ,
1                                           read manga
2                                                    -
3    need send ' em account tomorrow . odd , even r...
4                                     add myspac ! ! !
Name: message to examine, dtype: object

In [ ]:
atributos = data["message to examine"]
#Vectorización del texto del atributo con el codificador CountVectorizer
vectorizer = CountVectorizer()
#Conversión de los atributos en valores numéricos
atributos = vectorizer.fit_transform(atributos)

#Lo mismo para el objetivo
objetivo = data["label"]
objetivo = vectorizer.fit_transform(objetivo)

#Separación del conjunto de entrenamiento y de prueba
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        atributos, objetivo,
        random_state=12345,
        test_size=.2,
        stratify=objetivo)

#Entrenamiento del modelo de Naive Bayes usando la instancia MultinomialNB
sentiment_detector = MultinomialNB(alpha=1.0)  # alpha es el parámetro de suavizado
sentiment_detector.fit(atributos_entrenamiento, objetivo_entrenamiento)

#Predicciones con el conjunto de prueba
predicciones = sentiment_detector.predict(atributos_prueba)
#Cálculo de la precisión del modelo
precision = sentiment_detector.score(atributos_prueba, objetivo_prueba)
print("La precisión del modelo desarrollado es", precision*100)
